In [1]:
!pip install transformers sentencepiece torch

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
base_model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

model_path = "/kaggle/input/nllb-eng-naga-model/results/checkpoint-2475"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model and Pretrained Tokenizer Loaded Successfully!")

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

Model and Pretrained Tokenizer Loaded Successfully!


In [3]:
new_lang_token = "<ng>"  
tokenizer.add_special_tokens({"additional_special_tokens": [new_lang_token]})
model.resize_token_embeddings(len(tokenizer))

target_lang_id = tokenizer.convert_tokens_to_ids(new_lang_token)
if target_lang_id is None:
    raise ValueError(f"Error: Target language token '{new_lang_token}' not found in tokenizer.")


In [4]:
def translate_english_to_nagamese(text):
    src_lang = "eng_Latn" 
    tgt_lang = new_lang_token

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    output_tokens = model.generate(**inputs, forced_bos_token_id=target_lang_id)
    return tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]


In [5]:
english_sentences = [
    "For God so loved the world that He gave His only Son, that whoever believes in Him should not perish but have eternal life.",
    "I am going to the market.",
    "Can you help me with this?",
    "The roads are difficult to travel during the rainy season.",
    "Do to others as you would have them do to you."
]

translations = [translate_english_to_nagamese(sentence) for sentence in english_sentences]

for eng, nag in zip(english_sentences, translations):
    print(f"English: {eng}\nNagamese: {nag}\n")


English: For God so loved the world that He gave His only Son, that whoever believes in Him should not perish but have eternal life.
Nagamese: Kilemane Isor itu prithibi ke eneka morom korise, Tai nijor laga ekjon-ekjon Chokra ke di dise, jun Taike biswas kore, harai najabo, kintu anonto jibon pabo.

English: I am going to the market.
Nagamese: Moi dukh te jai ase.

English: Can you help me with this?
Nagamese: Apni moike modot koribo?

English: The roads are difficult to travel during the rainy season.
Nagamese: Rasta te berabole bisi digdar ase pani laga homoi te.

English: Do to others as you would have them do to you.
Nagamese: Aru jineka tumi khan dusra khan ke koribole itcha ase, itu nisena tumi bi dusra khan ke bi koribi.

